<a href="https://colab.research.google.com/github/Spandana0603/Credit-Card-Fraud-Detection-Using-Machine-Learning/blob/main/UI_Design_and_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %% [code]
# Colab + virtualenv + Rasa setup
# Update packages (may take a bit)
!apt-get update -y && apt-get install -y python3.8 python3.8-venv git -y

# Create a dedicated virtualenv for Rasa to avoid colab conflicts
!python3.8 -m venv /usr/local/rasa_venv

# Activate the virtualenv and upgrade pip, setuptools, wheel
!bash -c "source /usr/local/rasa_venv/bin/activate && python -m pip install --upgrade pip setuptools wheel"

# Install Rasa 3.1.x and dependencies inside the venv
# (Rasa and some deps may take a while to install)
!bash -c "source /usr/local/rasa_venv/bin/activate && pip install rasa==3.1.0 websockets==10.4 'sqlalchemy<2.0'"

# Install other helper libs: streamlit, pyngrok, psycopg2-binary, Flask for endpoints
!bash -c "source /usr/local/rasa_venv/bin/activate && pip install streamlit pyngrok psycopg2-binary flask==2.1.3"

# Show python & rasa versions to verify
!bash -c "source /usr/local/rasa_venv/bin/activate && python --version && rasa --version"


# %% [code]
# Create a folder for the project and move into it
!mkdir -p /content/rasa_healthcare_bot
%cd /content/rasa_healthcare_bot


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://cli.github.com/packages stable/main amd64 Packages [347 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,064 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ub

In [4]:
# Write config.yml
config_yaml = """
language: en

pipeline:
  - name: WhitespaceTokenizer
  - name: RegexFeaturizer
  - name: LexicalSyntacticFeaturizer
  - name: CountVectorsFeaturizer
  - name: DIETClassifier
    epochs: 100
  - name: EntitySynonymMapper
  - name: ResponseSelector
    epochs: 100

policies:
  - name: MemoizationPolicy
  - name: RulePolicy
  - name: UnexpecTEDIntentPolicy
    max_history: 5
    epochs: 100
"""

with open("config.yml", "w") as f:
    f.write(config_yaml.strip())
print("config.yml created")

# Write nlu.yml
import os
os.makedirs("data", exist_ok=True)

nlu_yaml = """
version: "3.1"

nlu:
- intent: greet
  examples: |
    - hello
    - hi
    - hey there
    - good morning
    - good evening

- intent: goodbye
  examples: |
    - bye
    - goodbye
    - see you later
    - take care

- intent: inform_symptom
  examples: |
    - I have chest pain
    - I feel shortness of breath
    - sore throat and headache
    - fever and chills
    - I’m feeling dizzy

- intent: provide_severity
  examples: |
    - mild
    - moderate
    - severe

- intent: provide_duration
  examples: |
    - 2 hours
    - 3 days
    - a week
    - since yesterday

- intent: provide_age
  examples: |
    - I am 25 years old
    - 40
    - age is 60
    - patient is 35

- intent: ask_appointment
  examples: |
    - I want to book an appointment
    - can I see a doctor?
    - I need an appointment
    - I’d like to book a slot

- intent: cancel_appointment
  examples: |
    - cancel my appointment
    - I want to cancel my booking
    - please cancel the appointment

- intent: check_availability
  examples: |
    - is the doctor available?
    - any free slots?
    - check availability for appointments

- intent: affirm
  examples: |
    - yes
    - yep
    - absolutely
    - sure

- intent: deny
  examples: |
    - no
    - nope
    - not now

- intent: out_of_scope
  examples: |
    - What’s the weather?
    - Tell me a joke
    - Who is the president?
"""

with open("data/nlu.yml", "w") as f:
    f.write(nlu_yaml.strip())
print("nlu.yml written successfully")


with open("data/nlu.yml", "w") as f:
    f.write(nlu_yaml.strip())
print("nlu.yml written successfully")

# Write stories.yml
stories_yaml = """
version: "3.1"

stories:
- story: symptom and triage
  steps:
    - intent: greet
    - action: utter_greet
    - intent: inform_symptom
    - action: utter_ask_severity
    - intent: provide_severity
    - action: utter_ask_duration
    - intent: provide_duration
    - action: utter_ask_age
    - intent: provide_age
    - action: action_triage
    - action: utter_ask_book_slot

- story: book appointment
  steps:
    - intent: ask_appointment
    - action: utter_ask_book_slot
    - intent: affirm
    - action: action_book_appointment
    - action: utter_appointment_confirmed
"""

with open("data/stories.yml", "w") as f:
    f.write(stories_yaml.strip())
print("stories.yml created")

# Write rules.yml
rules_yaml = """
version: "3.1"

rules:
- rule: greet then ask symptom
  steps:
  - intent: greet
  - action: utter_greet

- rule: fallback
  steps:
  - intent: out_of_scope
  - action: utter_default

- rule: fallback action
  steps:
  - intent: out_of_scope
  - action: action_default_fallback
"""

with open("rules.yml", "w") as f:
    f.write(rules_yaml.strip())
print("rules.yml created")

config.yml created
nlu.yml written successfully
nlu.yml written successfully
stories.yml created
rules.yml created


In [5]:
domain_yaml = """
version: "3.1"

intents:
  - greet
  - goodbye
  - inform_symptom
  - provide_severity
  - provide_duration
  - provide_age
  - ask_appointment
  - cancel_appointment
  - check_availability
  - affirm
  - deny
  - out_of_scope

entities:
  - symptom
  - severity
  - duration
  - age
  - appointment_slot
  - appointment_id

slots:
  symptoms:
    type: list
  severity:
    type: text
  duration:
    type: text
  age:
    type: float
  triage_decision:
    type: text
  appointment_slot:
    type: text
  appointment_id:
    type: text

responses:
  utter_greet:
    - text: "Hello! How can I assist you today? Please tell me your symptoms."
  utter_ask_severity:
    - text: "How severe are your symptoms? Mild, moderate, or severe?"
  utter_ask_duration:
    - text: "How long have you been experiencing these symptoms?"
  utter_ask_age:
    - text: "Can you please tell me your age?"
  utter_ask_book_slot:
    - text: "Would you like to book an appointment? If yes, please tell me your preferred time slot."
  utter_appointment_confirmed:
    - text: "Your appointment has been confirmed. Thank you!"
  utter_default:
    - text: "Sorry, I couldn't understand that. Can you please rephrase?"

actions:
  - action_triage
  - action_book_appointment
  - action_check_availability
  - action_cancel_appointment
  - action_handover_to_nurse
  - action_default_fallback
"""

with open("domain.yml", "w") as f:
    f.write(domain_yaml.strip())
print("domain.yml created")


domain.yml created


In [6]:
!pip install rasa-sdk


  Using cached Sanic_Cors-2.2.0-py2.py3-none-any.whl.metadata (7.8 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of googleapis-common-protos to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.9 MB/s eta 0:00:00
Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6

In [7]:
# actions.py - Custom Rasa actions for triage, booking, etc.
from typing import Any, Text, Dict, List
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
from rasa_sdk.events import SlotSet
import sqlite3
import uuid
from datetime import datetime

DB = "appointments.db"

# --- Database Initialization ---
def init_db():
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS appointments (
                   id TEXT PRIMARY KEY,
                   patient_age REAL,
                   symptoms TEXT,
                   triage TEXT,
                   slot TEXT,
                   created_at TEXT
               )""")
    conn.commit()
    conn.close()

# --- Booking Logic ---
def book_appointment(patient_age, symptoms, triage, slot):
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    appt_id = str(uuid.uuid4())[:8]
    created_at = datetime.utcnow().isoformat()
    c.execute("INSERT INTO appointments (id, patient_age, symptoms, triage, slot, created_at) VALUES (?, ?, ?, ?, ?, ?)",
              (appt_id, patient_age, symptoms, triage, slot, created_at))
    conn.commit()
    conn.close()
    return appt_id

def cancel_appointment_by_id(appointment_id):
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    c.execute("DELETE FROM appointments WHERE id = ?", (appointment_id,))
    affected = c.rowcount
    conn.commit()
    conn.close()
    return affected

def check_availability(slot_text):
    MAX_PER_SLOT = 3
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    c.execute("SELECT COUNT(*) FROM appointments WHERE slot = ?", (slot_text,))
    count = c.fetchone()[0]
    conn.close()
    return count < MAX_PER_SLOT

# --- Initialize DB on import ---
init_db()

# --- Simple Triage Logic ---
def simple_triage(symptoms: List[Text], severity: Text, duration: Text, age: float) -> Text:
    emergency_keywords = {"chest pain", "shortness of breath", "severe bleeding", "unconscious", "severe burn"}
    urgent_keywords = {"fever", "high fever", "severe headache", "confusion", "dizziness"}
    symptoms_lower = " ".join(symptoms).lower()

    if any(k in symptoms_lower for k in emergency_keywords) or (severity and severity.lower() == "severe"):
        return "urgent"
    if any(k in symptoms_lower for k in urgent_keywords) or (severity and severity.lower() == "moderate"):
        return "gp"
    if age and age >= 65 and ("fever" in symptoms_lower or "fall" in symptoms_lower):
        return "gp"
    return "selfcare"

# --- Actions ---
class ActionTriage(Action):
    def name(self) -> Text:
        return "action_triage"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
        symptoms = []
        events = tracker.latest_message.get('entities', [])
        for e in events:
            if e.get('entity') == 'symptom':
                symptoms.append(e.get('value'))

        if not symptoms:
            last_text = tracker.latest_message.get('text', "")
            symptoms = [last_text]

        severity = tracker.get_slot("severity")
        duration = tracker.get_slot("duration")
        age = tracker.get_slot("age")

        try:
            age_val = float(age) if age else None
        except:
            age_val = None

        triage_decision = simple_triage(symptoms, severity, duration, age_val)
        dispatcher.utter_message(text=f"Triage decision: {triage_decision}.")
        return [
            SlotSet("symptoms", symptoms),
            SlotSet("severity", severity),
            SlotSet("duration", duration),
            SlotSet("age", age_val),
            SlotSet("triage_decision", triage_decision)
        ]

class ActionBookAppointment(Action):
    def name(self) -> Text:
        return "action_book_appointment"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
        slot = tracker.get_slot("appointment_slot")
        if not slot:
            slot = tracker.latest_message.get('text')

        symptoms = tracker.get_slot("symptoms") or []
        age = tracker.get_slot("age") or None
        triage = tracker.get_slot("triage_decision") or "gp"

        if not check_availability(slot):
            dispatcher.utter_message(text="Sorry, that slot is fully booked. Please try another time.")
            return []

        appt_id = book_appointment(age, ",".join(symptoms), triage, slot)
        dispatcher.utter_message(text=f"Your appointment is booked for {slot}. Booking ID: {appt_id}")
        return [SlotSet("appointment_id", appt_id)]

class ActionCheckAvailability(Action):
    def name(self) -> Text:
        return "action_check_availability"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
        slot = tracker.latest_message.get('text')
        if check_availability(slot):
            dispatcher.utter_message(text=f"Slot {slot} is available.")
        else:
            dispatcher.utter_message(text=f"Slot {slot} is FULL. Try another time.")
        return []

class ActionCancelAppointment(Action):
    def name(self) -> Text:
        return "action_cancel_appointment"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
        appt_id = tracker.get_slot("appointment_id")
        if not appt_id:
            appt_id = tracker.latest_message.get('text')

        affected = cancel_appointment_by_id(appt_id)
        if affected:
            dispatcher.utter_message(text=f"Appointment {appt_id} cancelled.")
        else:
            dispatcher.utter_message(text=f"Could not find appointment {appt_id}. Please check your ID.")
        return []

class ActionDefaultFallback(Action):
    def name(self) -> Text:
        return "action_default_fallback"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
        dispatcher.utter_message(text="Sorry, I couldn't understand that. Let me connect you to a human assistant.")
        return []


In [ ]:
with open("endpoints.yml", "w") as f:
    f.write("""
action_endpoint:
  url: "http://localhost:5055/webhook"
""".strip())

print("endpoints.yml created successfully.")


endpoints.yml created successfully.


In [8]:
%cd /content/rasa_healthcare_bot


/content/rasa_healthcare_bot


In [9]:
!source /usr/local/rasa_venv/bin/activate && rasa train --debug


/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/core/tracker_store.py:830: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
2025-10-02 12:56:10 DEBUG    rasa.telemetry  - Could not read telemetry settings from configuration file: Configuration 'metrics' key not found.
(0lqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqk(B
(0x(B Rasa Open Source reports anonymous usage telemetry to help improve the product (0x(B
(0x(B for all its users.                                                            

In [10]:
!bash -c "source /usr/local/rasa_venv/bin/activate && nohup rasa run actions --port 5055 > actions_server.log 2>&1 &"
!sleep 2
!tail -n 30 actions_server.log || true


/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/core/tracker_store.py:830: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/shared/utils/validation.py:131: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/rasa_venv/lib/python3.8/site-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit 

In [11]:
!bash -c "source /usr/local/rasa_venv/bin/activate && nohup rasa run --enable-api --cors '*' > rasa_server.log 2>&1 &"
!sleep 4
!tail -n 40 rasa_server.log || true


    from rasa.core.agent import Agent
  File "/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/core/agent.py", line 23, in <module>
    from rasa.core.policies.policy import PolicyPrediction
  File "/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/core/policies/policy.py", line 26, in <module>
    from rasa.core.featurizers.tracker_featurizers import TrackerFeaturizer
  File "/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/core/featurizers/tracker_featurizers.py", line 12, in <module>
    from rasa.core.featurizers.single_state_featurizer import SingleStateFeaturizer
  File "/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/core/featurizers/single_state_featurizer.py", line 7, in <module>
    from rasa.nlu.extractors.extractor import EntityTagSpec
  File "/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/nlu/extractors/extractor.py", line 30, in <module>
    import rasa.utils.train_utils
  File "/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/utils/train_u

In [12]:
!bash -c "source /usr/local/rasa_venv/bin/activate && rasa shell"


/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/core/tracker_store.py:830: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/shared/utils/validation.py:131: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/rasa_venv/lib/python3.8/site-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit 

In [13]:
!bash -c "source /usr/local/rasa_venv/bin/activate && rasa shell"


/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/core/tracker_store.py:830: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/rasa_venv/lib/python3.8/site-packages/rasa/shared/utils/validation.py:131: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/rasa_venv/lib/python3.8/site-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit 

In [14]:
!source /usr/local/rasa_venv/bin/activate


In [1]:
# Create a fresh virtualenv with Python 3.7 or 3.8
!python3.7 -m venv rasa_venv
!source rasa_venv/bin/activate

# Downgrade pip to improve compatibility
!pip install pip==20.2 setuptools==49.6.0 wheel==0.36.2

# Now try installing the old version of Rasa
!pip install rasa==1.10.2


/bin/bash: line 1: python3.7: command not found
/bin/bash: line 1: rasa_venv/bin/activate: No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/pip", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/autocompletion.py", line 9, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/main_parser.py", line 7, in <module>
    from pip._internal.cli import cmdoptions
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/cmdoptions.py", line 18, in <module>
    from distutils.util import strtobool
ModuleNotFoundError: No module named 'distutils'
Traceback (most recent call last):
  File "/usr/local/bin/pip", line 5, in <module>


In [ ]:
!apt-get update
!apt-get install -y python3-distutils


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,632 B in 2s (2,290 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [2]:
# Remove old venv
!rm -rf rasa_venv

# Install Python 3.8 (you can also try 3.7)
!sudo apt-get update -y
!sudo apt-get install python3.8 python3.8-venv python3.8-dev -y

# Create venv with python3.8
!python3.8 -m venv rasa_venv

# Upgrade pip inside the new venv
!rasa_venv/bin/python3.8 -m pip install --upgrade pip setuptools wheel

# Install rasa inside the venv
!rasa_venv/bin/python3.8 -m pip install rasa

# Check if rasa installed correctly
!rasa_venv/bin/rasa --version


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
!source rasa_venv/bin/activate


In [ ]:
!rasa_venv/bin/rasa init --no-prompt
!rasa_venv/bin/rasa train
!rasa_venv/bin/rasa shell


/content/rasa_venv/lib/python3.8/site-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/content/rasa_venv/lib/python3.8/site-packages/rasa/shared/utils/validation.py:134: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/content/rasa_venv/lib/python3.8/site-packages/pkg_resources/__init__.py:3117: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit

In [3]:
!rasa_venv/bin/rasa train --debug


/content/rasa_venv/lib/python3.8/site-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/content/rasa_venv/lib/python3.8/site-packages/rasa/shared/utils/validation.py:134: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/content/rasa_venv/lib/python3.8/site-packages/pkg_resources/__init__.py:3117: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit

In [1]:
import subprocess
import time
import requests

# Paths
RASA_EXEC = '/content/rasa_venv/bin/rasa'
MODEL_PATH = 'models/20250917-164011-silly-banquette.tar.gz'

actions_server = subprocess.Popen(
    [RASA_EXEC, "run", "actions", "--port", "5055"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# --- Start Rasa Core Server ---
rasa_server = subprocess.Popen(
    [
        RASA_EXEC, "run",
        "--model", MODEL_PATH,
        "--enable-api",
        "--cors", "*",
        "--port", "5005"
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# --- Wait until Rasa server is ready ---
def server_ready(url="http://localhost:5005/status", timeout=60):
    start = time.time()
    while True:
        try:
            res = requests.get(url)
            if res.status_code == 200:
                print("Rasa server is up!")
                return
        except requests.exceptions.ConnectionError:
            pass
        if time.time() - start > timeout:
            raise RuntimeError("Server did not start in time.")
        time.sleep(2)

server_ready()

# --- Function to send messages ---
def talk_to_rasa(msg, sender_id="user"):
    endpoint = "http://localhost:5005/webhooks/rest/webhook"
    payload = {"sender": sender_id, "message": msg}
    res = requests.post(endpoint, json=payload)
    return res.json()

# --- Test conversation ---
print(talk_to_rasa("Hey!"))
print(talk_to_rasa("Tell me a joke"))


FileNotFoundError: [Errno 2] No such file or directory: '/content/rasa_venv/bin/rasa'

In [4]:
!/content/rasa_venv/bin/python -m rasa_sdk --actions actions --port 5055 &

/bin/bash: line 1: /content/rasa_venv/bin/python: No such file or directory


In [3]:
!/content/rasa_venv/bin/rasa train --debug

/bin/bash: line 1: /content/rasa_venv/bin/rasa: No such file or directory


In [2]:
# Recreate config.yml
config_yaml = """
language: en

pipeline:
  - name: WhitespaceTokenizer
  - name: RegexFeaturizer
  - name: LexicalSyntacticFeaturizer
  - name: CountVectorsFeaturizer
  - name: DIETClassifier
    epochs: 100
  - name: EntitySynonymMapper
  - name: ResponseSelector
    epochs: 100

policies:
  - name: MemoizationPolicy
  - name: RulePolicy
  - name: UnexpecTEDIntentPolicy
    max_history: 5
    epochs: 100
"""

with open("config.yml", "w") as f:
    f.write(config_yaml.strip())
print("config.yml created")

# Recreate nlu.yml
import os
os.makedirs("data", exist_ok=True)

nlu_yaml = """
version: "3.1"

nlu:
- intent: greet
  examples: |
    - hello
    - hi
    - hey there
    - good morning
    - good evening

- intent: goodbye
  examples: |
    - bye
    - goodbye
    - see you later
    - take care

- intent: inform_symptom
  examples: |
    - I have chest pain
    - I feel shortness of breath
    - sore throat and headache
    - fever and chills
    - I’m feeling dizzy

- intent: provide_severity
  examples: |
    - mild
    - moderate
    - severe

- intent: provide_duration
  examples: |
    - 2 hours
    - 3 days
    - a week
    - since yesterday

- intent: provide_age
  examples: |
    - I am 25 years old
    - 40
    - age is 60
    - patient is 35

- intent: ask_appointment
  examples: |
    - I want to book an appointment
    - can I see a doctor?
    - I need an appointment
    - I’d like to book a slot

- intent: cancel_appointment
  examples: |
    - cancel my appointment
    - I want to cancel my booking
    - please cancel the appointment

- intent: check_availability
  examples: |
    - is the doctor available?
    - any free slots?
    - check availability for appointments

- intent: affirm
  examples: |
    - yes
    - yep
    - absolutely
    - sure

- intent: deny
  examples: |
    - no
    - nope
    - not now

- intent: out_of_scope
  examples: |
    - What’s the weather?
    - Tell me a joke
    - Who is the president?
"""

with open("data/nlu.yml", "w") as f:
    f.write(nlu_yaml.strip())
print("data/nlu.yml written successfully")

# Recreate stories.yml
stories_yaml = """
version: "3.1"

stories:
- story: symptom and triage
  steps:
    - intent: greet
    - action: utter_greet
    - intent: inform_symptom
    - action: utter_ask_severity
    - intent: provide_severity
    - action: utter_ask_duration
    - intent: provide_duration
    - action: utter_ask_age
    - intent: provide_age
    - action: action_triage
    - action: utter_ask_book_slot

- story: book appointment
  steps:
    - intent: ask_appointment
    - action: utter_ask_book_slot
    - intent: affirm
    - action: action_book_appointment
    - action: utter_appointment_confirmed
"""

with open("data/stories.yml", "w") as f:
    f.write(stories_yaml.strip())
print("data/stories.yml created")

# Recreate rules.yml
rules_yaml = """
version: "3.1"

rules:
- rule: greet then ask symptom
  steps:
  - intent: greet
  - action: utter_greet

- rule: fallback
  steps:
  - intent: out_of_scope
  - action: utter_default

- rule: fallback action
  steps:
  - intent: out_of_scope
  - action: action_default_fallback
"""

with open("rules.yml", "w") as f:
    f.write(rules_yaml.strip())
print("rules.yml created")

# Recreate domain.yml
domain_yaml = """
version: "3.1"

intents:
  - greet
  - goodbye
  - inform_symptom
  - provide_severity
  - provide_duration
  - provide_age
  - ask_appointment
  - cancel_appointment
  - check_availability
  - affirm
  - deny
  - out_of_scope

entities:
  - symptom
  - severity
  - duration
  - age
  - appointment_slot
  - appointment_id

slots:
  symptoms:
    type: list
  severity:
    type: text
  duration:
    type: text
  age:
    type: float
  triage_decision:
    type: text
  appointment_slot:
    type: text
  appointment_id:
    type: text

responses:
  utter_greet:
    - text: "Hello! How can I assist you today? Please tell me your symptoms."
  utter_ask_severity:
    - text: "How severe are your symptoms? Mild, moderate, or severe?"
  utter_ask_duration:
    - text: "How long have you been experiencing these symptoms?"
  utter_ask_age:
    - text: "Can you please tell me your age?"
  utter_ask_book_slot:
    - text: "Would you like to book an appointment? If yes, please tell me your preferred time slot."
  utter_appointment_confirmed:
    - text: "Your appointment has been confirmed. Thank you!"
  utter_default:
    - text: "Sorry, I couldn't understand that. Can you please rephrase?"

actions:
  - action_triage
  - action_book_appointment
  - action_check_availability
  - action_cancel_appointment
  - action_handover_to_nurse
  - action_default_fallback
"""

with open("domain.yml", "w") as f:
    f.write(domain_yaml.strip())
print("domain.yml created")

config.yml created
data/nlu.yml written successfully
data/stories.yml created
rules.yml created
domain.yml created


In [ ]:
import requests

def send_message(message, sender="user"):
    url = "http://localhost:5005/webhooks/rest/webhook"
    payload = {"sender": sender, "message": message}
    response = requests.post(url, json=payload)
    return response.json()

while True:
    msg = input("You: ")
    if msg.lower() in ["exit", "quit"]:
        break
    responses = send_message(msg)
    for r in responses:
        print("Bot:", r.get("text"))
